# E91 Quantum Key Distribution

This notebook demonstrates the E91 Quantum Key Distribution protocol from Chapter 10, Key Applications of Entanglement.

[![Open In Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/Polaris-QCI/Quantum-Computing-and-Information-ed2/main?labpath=Qiskit%2F10-Key-Applications-of-Entanglement%2Fe91-quantum-key-distribution-demo.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Polaris-QCI/Quantum-Computing-and-Information-ed2/blob/main/Qiskit/10-Key-Applications-of-Entanglement/e91-quantum-key-distribution-demo.ipynb)

## 0. Setup

The cells below install the packages and import the libraries needed by this demo.

In [ ]:
#@title Install Qiskit and other dependencies
%pip install pylatexenc qiskit qiskit-aer qiskit-ibm-runtime

In [2]:
# Imports

import random
from IPython.display import display, Markdown
from math import cos, sin, pi as π
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.circuit import ClassicalRegister, QuantumCircuit, QuantumRegister

## 1. E91 bit

The next cell implements the E91 protocol for a single potential key bit. 

In [3]:
def e91_bit():
    q = [q0, q1] = QuantumRegister(2)
    c = ClassicalRegister(2, 'c')
    circuit = QuantumCircuit(q, c)

    # Generate the Ψ⁻ Bell state
    circuit.x(q0)
    circuit.x(q1)
    circuit.h(q0)
    circuit.cx(q0, q1)

    # Alice's basis choices
    # a1 = Z (no rotation needed)
    # a2 = W = 1/√2(Z+X)
    # a3 = X
    rand = random.randint(0, 2)
    if rand == 0:  # Z basis
        alice_basis = "a1"
    if rand == 1:  # W basis
        alice_basis = "a2"
        circuit.ry(-π/4, q0)
    elif rand == 2:  # X basis
        alice_basis = "a3"
        circuit.h(q0)

    # Bob's basis choices
    # b1 = W = 1/√2(Z+X)
    # b2 = X
    # b3 = V = 1/√2(Z-X)
    rand = random.randint(0, 2)
    if rand == 0:  # W basis
        bob_basis = "b1"
        circuit.ry(-π/4, q1)
    elif rand == 1:  # X basis
        bob_basis = "b2"
        circuit.h(q1)
    else:  # V basis
        bob_basis = "b3"
        circuit.ry(π/4, q1)

    circuit.measure(q, c)

    simulator = AerSimulator()
    sampler = Sampler(simulator)
    job = sampler.run([circuit], shots=1)
    job_result = job.result()
    result = next(iter(job_result[0].data.c.get_counts().keys()))
    alice_result = result[0]
    bob_result = result[1]
    return {
        "alice_basis": alice_basis,
        "bob_basis": bob_basis,
        "alice_result": alice_result,
        "bob_result": bob_result,
    }


In [6]:
output =  "| # | Alice Basis | Bob Basis | Use for Key | Measurement Result | Use for Checking |\n"
output += "|---|-------------|-----------|-------------|----------------|--------------|\n"
for i in range(20):
    bit = e91_bit()
    alice_basis = bit['alice_basis']
    bob_basis = bit['bob_basis']
    use_for_key = "✓" if alice_basis == "a2" and bob_basis == "b1" or alice_basis == "a3" and bob_basis == "b2" else " "
    measurement_result = f"**{bit['alice_result']}**" if use_for_key == "✓" else bit['alice_result']
    use_for_checking = "✓" if alice_basis != "a2" and bob_basis != "b2" else "✗"
    output += f"| {i+1} | {alice_basis} | {bob_basis} | {use_for_key} | {measurement_result} | {use_for_checking} |\n"
display(Markdown(output))

| # | Alice Basis | Bob Basis | Use for Key | Measurement Result | Use for Checking |
|---|-------------|-----------|-------------|----------------|--------------|
| 1 | a2 | b2 |   | 0 | ✗ |
| 2 | a2 | b3 |   | 0 | ✗ |
| 3 | a1 | b2 |   | 0 | ✗ |
| 4 | a2 | b2 |   | 0 | ✗ |
| 5 | a3 | b2 | ✓ | **0** | ✗ |
| 6 | a3 | b1 |   | 0 | ✓ |
| 7 | a3 | b3 |   | 0 | ✓ |
| 8 | a3 | b3 |   | 1 | ✓ |
| 9 | a2 | b1 | ✓ | **1** | ✗ |
| 10 | a1 | b2 |   | 0 | ✗ |
| 11 | a1 | b2 |   | 1 | ✗ |
| 12 | a3 | b2 | ✓ | **1** | ✗ |
| 13 | a1 | b3 |   | 1 | ✓ |
| 14 | a3 | b2 | ✓ | **1** | ✗ |
| 15 | a3 | b1 |   | 0 | ✓ |
| 16 | a1 | b1 |   | 1 | ✓ |
| 17 | a3 | b1 |   | 1 | ✓ |
| 18 | a2 | b3 |   | 1 | ✗ |
| 19 | a2 | b3 |   | 0 | ✗ |
| 20 | a2 | b1 | ✓ | **0** | ✗ |
